# Implementing the correlation ratio

In [ ]:
# Don't change this cell; just run it.
import numpy as np  # The array library.

import pandas as pd

# Safe setting for Pandas.  Needs Pandas version >= 1.5.
pd.set_option('mode.copy_on_write', True)

## About the correlation ratio

The correlation is a statistic for the situation where you have some continuous variable, with numbers, and you have another variable that contains labels, that identify groups to which each row belongs.   See below for a worked example.

Have a look at the [Wikipedia page on correlation ratio](https://en.wikipedia.org/wiki/Correlation_ratio) for background.

It is not widely used, and for that reason, there is no standard implementation of this statistic.

Here you will do you own implementation.

## The data

`wine.csv` has a table of data, for which each row corresponds to a particular wine, and the columns correspond to measure for that wine.   The first column gives the class of the wine, where the class corresponds to a particular [cultivar](https://en.wikipedia.org/wiki/Cultivar) - a grape variety.  Thus all rows with wine class 1 come from one cultivar, from class 2 another, and from class three, a third.

See [the dataset page](https://archive.ics.uci.edu/ml/datasets/Wine) for more detail.

In [ ]:
wine = pd.read_csv('wine.csv')
wine.head()

## Introduction to the correlation ratio

Here we get into a little terminology and notation from the Wikipedia page.  Don't worry if this is hard to follow, you can work out the calculation from the code below.

Imagine you have $n$ observations, broken down into $g$ categories, but their label.  For example, consider the DataFrame above, and look only at the `Alcohol` values, and the grape `Class`:

In [ ]:
categories = wine['Class']
values = wine['Alcohol']
# Display categories and values as a DataFrame
cat_vals = pd.DataFrame({'categories': categories, 'values': values})
cat_vals

In [ ]:
n = len(values)
n

The $n$ observations are in $c$ categories.  Call these $c$ categories *levels*.

In [ ]:
levels = categories.unique()
levels

In [ ]:
c = len(levels)
c

## The notation in the Wikipedia page

**This section is optional**.  It goes through the mathematical notation in the Wikipedia page.  You don't need to understand this section to understand the exercise, but it relates the calculation to the mathematical notation on the Wikipedia page.  If you don't follow this section, consider going ahead to the next section to see how to calculate the correlation ratio in code, then come back here to see how that relates to the notation.

Write the mean of group 2 as $\bar{y_2}$, and, in general, write the mean of group $x$ as $\bar{y_x}$.  Here are the observations in our actual group 2:

In [ ]:
values_in_group_2 = values[categories == 2]
values_in_group_2

Write the number of values in group 2 as $n_2$, and, in general, write the number of values in group $x$ as $n_x$.

In [ ]:
n_2 = len(values_in_group_2)
n_2

Write the sequence of values in group 2 as:

$$
[y_{2,1}, y_{2,2}, y_{2,3}, ..., y_{2,n_2}]
$$

where $y_{2,3}$ means the 3rd $y$ value in group 2.

More generally, write the sequence of values in group $x$ as:

$$
[y_{x,1}, y_{x,2}, y_{x,3}, ..., y_{x,n_x}]
$$

As usual, we calculate the mean for group 2 (generally, group $x$) by:

* Adding up the values in group 2 (generally, group $x$)
* Dividing by $n_2$ (generally, $n_x$)

Write the result of adding up the values in group 2 as:

$$
\sum_{i=1}^{i=n_2}{y_{2,i}}
$$

The $\sum$ is the Greek symbol for capital S, and means "the sum of".   Read the line above as "the sum of all the values in group 2, starting with the first ($i=1$) and going up to the last ($i=n_2$)".

Because we nearly always do sum from the first through the last value, we can also write that with the shorthand:

$$
\sum_i{y_{2,i}}
$$

Then the mean for the values in group 2 is given by:

$$
\bar{y}_2 = \frac{\sum_i{y_{2,i}}} {n_2}
$$

In code that is:

In [ ]:
y_bar_2 = values_in_group_2.sum() / n_2
y_bar_2

In general, the mean for group $x$ is:

$$
\bar{y}_x = \frac{\sum_i{y_{x,i}}} {n_x}
$$

The following is (slightly edited) from Wikipedia:

> Suppose each observation is $y_{x,i}$ where $x$ indicates the category that observation is in and $i$ is the label of the particular observation.  Let $n_x$ be the number of observations in category $x$ and

$$
\bar{y}_x=\frac{\sum_i y_{x,i}}{n_x}
$$

> and

$$
\bar{y}=\frac{\sum_x n_x \bar{y}_x}{\sum_x n_x}
$$

> where $\bar{y}_x$ is the mean of the category $x$ and $\bar{y}$ is the mean of the whole population. The correlation ratio $\eta$ is defined as to satisfy:

$$
\eta^2 = \frac{\sum_x n_x (\bar{y}_x-\bar{y})^2}{\sum_{x,i} (y_{x,i}-\bar{y})^2}
$$

Notice that this expression:

$$
\bar{y}=\frac{\sum_x n_x \bar{y}_x}{\sum_x n_x}
$$

is just a way of writing the overall mean, ignoring groups, because these:

$$
n_x \bar{y}_x
$$

are just the sums of the observations in the groups, so the overall expression is just the sum of all the observations, divided by the overall number of observations. Therefore, in code we can just calculate that result with:

In [ ]:
y_bar = values.sum() / len(values)
y_bar

Or:

In [ ]:
y_bar = values.mean()
y_bar

**Don't worry if you don't follow the maths above**.   It's just to help you get used to the way the places like Wikipedia will describe statistical and mathematical formulae.   You'll see the calculation in action below, in code.  But you may want to look back and forth between the explanation above and the code to see how they match up.

## Long-hand calculation of the correlation ratio

We want to calculate the correlation ratio for the numerical column `Alcohol` (the alcohol level of the wine), given the labels in `Class`.

This is how we would do that by hand, following the formula in the Wikipedia page.

In [ ]:
y_bar = values.mean()  # Overall mean
overall_sum_of_squares = ((values - y_bar) ** 2).sum()
levels = categories.dropna().unique()
n_levels = len(levels)
# Calculate the entries in the group sum of squares
group_sums_of_squares = np.zeros(n_levels)
# For each level
for group_no in np.arange(n_levels):
    level = levels[group_no]
    # Select values corresponding to this level. 
    is_in_level = categories == level
    level_values = values[is_in_level]
    # Get the n_x times the (mean for this group minus overall mean) squared.
    n_in_level = len(level_values)
    group_sos = n_in_level * (level_values.mean() - y_bar) ** 2
    # Store in entries for group sum of squares.
    group_sums_of_squares[group_no] = group_sos
top_of_stat = np.sum(group_sums_of_squares)  # Numerator of correlation ratio
bottom_of_stat = np.sum((values - y_bar) ** 2)  # Denominator
eta_long_way = np.sqrt(top_of_stat / bottom_of_stat)
eta_long_way

While we're here, let's get another dataset.  For this dataset, each row is a patient, each column is some measure of blood, urine, or clinical feature for that patient.  The column `Coronary Artery Disease` specifies whether the patient qualified for a diagnosis of heart vessel disease ('yes') or not ('no').  The column `Hemoglobin`  gives the blood level of the protein that carries oxygen around the body.

In [ ]:
ckd = pd.read_csv('ckd.csv')
ckd.head()

Lastly, here is the example from Wikipedia:

In [ ]:
eg_df = pd.DataFrame()
eg_df['categories'] = np.repeat(['Algebra', 'Geometry', 'Statistics'], [5, 4, 6])
eg_df['values'] =  [45, 70, 29, 15, 21, 40, 20, 30,42, 65, 95, 80, 70, 85, 73]
eg_eta = np.sqrt(6780 / 9640)  # Calculation from Wikipedia
eg_df

## Your job

Write a function `correlation_ratio`, that returns this value for any data frame, for a given numerical column (given by the column name) and label column (given by a column name):

In [ ]:
    # Your code here
    ...
    return ...

In [ ]:
# Test your function by replicating the analysis above.
# When your function is correct, you should be able to run this cell without error
assert np.isclose(correlation_ratio('Alcohol', 'Class', wine), eta_long_way)

In [ ]:
# Test it on another column.
assert np.isclose(correlation_ratio('Malic Acid', 'Class', wine), 0.544857081967286)

In [ ]:
# Test it on another dataset.
assert np.isclose(correlation_ratio('Hemoglobin', 'Coronary Artery Disease', ckd), 0.3787772107398905)

In [ ]:
# Test it on Wikipedia dataset.
assert np.isclose(correlation_ratio('values', 'categories', eg_df), eg_eta)

For extra points, document the function correctly using the [Numpy Docstring Standard](https://numpydoc.readthedocs.io/en/latest/format.html#docstring-standard)

## Done.

Congratulations, you're done with the assignment!  Be sure to:

- **run all the tests** (with Restart and Run All)
- **Save and Checkpoint** from the `File` menu.